### First-Mile Problem

Consider:
- 30 requests to transit stations
- 2 dummy nodes to depart and return
- time in minutes relative to the vehicle strating time
- Assume every vehicle starts at same time and works for 

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

### 1. Data

In [164]:
# Load
sd = pd.read_csv('service_duration.csv')
df_97 = pd.read_csv('reg2_97.csv')
df_95 = pd.read_csv('reg2_95.csv')
df_123 = pd.read_csv('reg2_123.csv')
travel_time = pd.read_csv('TravelTime_dp.csv')

df_95 = df_95.iloc[:-10]
df_97 = df_97.iloc[:-10]
# df_123 = df_123.iloc[:2]

df = pd.concat([df_95, df_97, df_123], axis=0)
df['trip_id'] = [i for i in range(1,len(df)+1)]

# set of nodes
P = [i for i in df.trip_id]
D = [i+len(df) for i in df.trip_id]
dummy = [0, 2*len(df)+1]
N = P + D + dummy

# demand of each trip
q_p = {i: df.loc[df['trip_id'] == i, 'k_i1'].iloc[0] for i in P }
q_d = {i: df.loc[df['trip_id'] == i-len(df), 'k_i2'].iloc[0] for i in D }
q_dd = {i: 0 for i in dummy}   # is it correct?
q = {**q_p, **q_d, **q_dd} 

# early serving time
e_p = {i: df.loc[df['trip_id'] == i, 'e_i1'].iloc[0] for i in P }
e_d = {i: df.loc[df['trip_id'] == i-len(df), 'e_i2'].iloc[0] for i in D }
e_dd = {i: 0 for i in dummy}   # the earliest possible from the vehicle start dispatch time
ea = {**e_p, **e_d, **e_dd} 

# latest serving time
M = 1e6
l_p = {i: df.loc[df['trip_id'] == i, 'l_i1'].iloc[0] for i in P }
l_d = {i: df.loc[df['trip_id'] == i-len(df), 'l_i2'].iloc[0] for i in D }
l_dd = {i: M for i in dummy}   # unrestricted value
l = {**l_p, **l_d, **l_dd} 

# Node-Trip ID
node_trip_p = {i: df.loc[df['trip_id'] == i, 'reg1'].iloc[0] for i in P }
node_trip_d = {i: df.loc[df['trip_id'] == i-len(df), 'reg2'].iloc[0] for i in D }
node_trip_dd = {i: i for i in dummy}
node_trip = {**node_trip_p, **node_trip_d, **node_trip_dd}

# travel time
A = [(i, j) for i in N for j in N if i != j]  # complete graph
tij = { (i,j): travel_time.iat[node_trip[i],node_trip[j]] for i,j in A}
# check - 0,101 and 101,0 must be 0


# maximum service time
r = 0.5
Lmax = {i: (1+r)*tij[(i,len(df)+i)] for i in P }

# service duration
# assume fix service duretion (loads and unload passenger)
s_pd = {i: 3 for i in P+D}
s_dd = {i: 0 for i in dummy}
s = {**s_pd, **s_dd}


# set of vehicles
m = 50
M = [i for i in range(1,m+1)]
Tm = {i: np.random.randint(100,400) for i in M}  # on average the ride-sharing only work 4-6 hours/day
Qmax = 4 # assume every vehicles is medium passenger cars (max 5 seats)

# weights
a1 = 0.6  
a2 = 0.4
a3 = 0.4

In [372]:
df.to_csv('large_network_data.csv')

In [166]:
print(P,D,N)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 0, 61]


In [167]:
2*len(df)+1

61

### 2. Problem Formulation

Step 1: each request is served by a vehicle

In [23]:
initial = []
for i in range(0, len(df)):
    temp = []
    temp.append(0)
    temp.append(df.iloc[i,8])
    temp.append(df.iloc[i,8]+30)
    temp.append(2*len(df)+1)
    
    initial.append(temp)
    
print(initial)
    
# print(obj_1(result))
# print(obj_2(result))
# print(obj_3(result))

[[0, 1, 31, 61], [0, 2, 32, 61], [0, 3, 33, 61], [0, 4, 34, 61], [0, 5, 35, 61], [0, 6, 36, 61], [0, 7, 37, 61], [0, 8, 38, 61], [0, 9, 39, 61], [0, 10, 40, 61], [0, 11, 41, 61], [0, 12, 42, 61], [0, 13, 43, 61], [0, 14, 44, 61], [0, 15, 45, 61], [0, 16, 46, 61], [0, 17, 47, 61], [0, 18, 48, 61], [0, 19, 49, 61], [0, 20, 50, 61], [0, 21, 51, 61], [0, 22, 52, 61], [0, 23, 53, 61], [0, 24, 54, 61], [0, 25, 55, 61], [0, 26, 56, 61], [0, 27, 57, 61], [0, 28, 58, 61], [0, 29, 59, 61], [0, 30, 60, 61]]


In [237]:
def initial(df):
    initial = []
    for i in range(0, len(df)):
        temp = []
        temp.append(0)
        temp.append(df.iloc[i,8])
        temp.append(df.iloc[i,8]+30)
        temp.append(61)
    
        initial.append(temp)
        
    return initial

In [233]:
#obj1
def obj_1(tour):
    obj = 0
    for j in range(len(tour)):
        tour_temp = tour[j]
        sum_time = 0
        for i in range(len(tour_temp)-1):
            sum_time += tij[tour_temp[i],tour_temp[i+1]]
            
        obj += sum_time
        
    return obj

In [276]:
#obj2
def obj_2(tour):
    obj = 0
    for j in range(len(tour)):
        tour_temp = tour[j]
        sum_time = 0
        destination = tour_temp[-2]
#         print(destination)
#         print(tour_temp)
#         print(tour_temp[0:len(tour_temp)-1])
        B = compute_time(tour_temp[0:len(tour_temp)-1])
#         print(B,B[-1])
        for m in range(0,len(B)-1):
            sum_time += B[-1] - B[m] - tij[tour_temp[m+1],destination]
#             print(tour_temp[m+1])
#             print(tij[tour_temp[m+1],destination])
#             print(sum_time)
            
        obj += sum_time
        
    return obj

In [235]:
#obj3
def obj_3(tour):
    obj = 0
    for j in range(len(tour)):
        tour_temp = tour[j]
        wait_time = 0
#         destination = tour_temp[-2]
# #         print(destination)
# #         print(tour_temp)
# #         print(tour_temp[0:len(tour_temp)-1])
        B = compute_time(tour_temp[0:len(tour_temp)-1])
#         print(B)
        for m in range(len(B)-1):
#             print(B[m], e_p[tour_temp[m+1]],q_p[tour_temp[m+1]])
            wait_time += (B[m] - e_p[tour_temp[m+1]])*q_p[tour_temp[m+1]]
#             print(wait_time)
#             print(tour_temp[m+1])
#             print(tij[tour_temp[m+1],destination])
#             print(sum_time)
            
        obj += wait_time
        
    return obj

In [236]:
#compute time
def compute_time(tour):
    B = []
    B_1m = max(tij[tour[0],tour[1]], e_p[tour[1]])
#     print(B_1m)
#     print(l_p[tour[1]])
    if B_1m > l_p[tour[1]]:
            a = 'infeasible!'
#             print(a)
            return a 
    else:
            B.append(B_1m)
            B_im = B_1m
    for i in range(1, len(tour)-2):
        B_jm = max(B_im + tij[tour[i],tour[i+1]] + s_pd[tour[i]], e_p[i+1])
#         print(B_jm)
        if B_jm > l_p[i+1]:
#             print(l_p[i+1])
            a = 'infeasible!'
#             print(a)
            return a 
        else:
            B.append(B_jm)
        B_im = B_jm
    
    
    B_jm = max(B_im + tij[tour[-2],tour[-1]] + s_pd[tour[-2]], e_d[tour[-1]])
    if B_jm > l_d[tour[-1]]:
#             print(l_p[i+1])
        a = 'infeasible!'
#         print(a)
        return a 
    else:
        B.append(B_jm)
        B_im = B_jm
            
    return B

Step 2: According to the destinations of the requests, divide the requests to different groups and preprocessing the network.

In [238]:
# Group 1
g_1 = df[df['reg2']==95]
print(g_1)

g_1_initial = initial(g_1)
print(g_1_initial)

   e_i1  l_i1  k_i1  reg1  e_i2  l_i2  k_i2  reg2  trip_id
0    40    57     2   126    60    77    -2    95        1
1    24    47     1    98    30    53    -1    95        2
2    15    50     1    79    33    68    -1    95        3
3    37    58     1    96    56    77    -1    95        4
4    11    43     3    29    29    61    -3    95        5
[[0, 1, 31, 61], [0, 2, 32, 61], [0, 3, 33, 61], [0, 4, 34, 61], [0, 5, 35, 61]]


In [239]:
# Group 2
g_2 = df[df['reg2']==97]
print(g_2)

g_2_initial = initial(g_2)
print(g_2_initial)

   e_i1  l_i1  k_i1  reg1  e_i2  l_i2  k_i2  reg2  trip_id
0    30    45     1   127    44    64    -1    97        6
1    16    45     2    33    36    70    -2    97        7
2    22    43     2    53    29    55    -2    97        8
3    30    52     2    48    46    73    -2    97        9
4    18    40     3   157    32    59    -3    97       10
5    20    48     2    49    26    59    -2    97       11
6    13    35     1    76    26    53    -1    97       12
7    22    50     3    57    27    60    -3    97       13
8    12    34     3    55    23    50    -3    97       14
9     7    38     3   153    26    62    -3    97       15
[[0, 6, 36, 61], [0, 7, 37, 61], [0, 8, 38, 61], [0, 9, 39, 61], [0, 10, 40, 61], [0, 11, 41, 61], [0, 12, 42, 61], [0, 13, 43, 61], [0, 14, 44, 61], [0, 15, 45, 61]]


In [240]:
# Group 3
g_3 = df[df['reg2']==123]
print(g_3)

g_3_initial = initial(g_3)
print(g_3_initial)

    e_i1  l_i1  k_i1  reg1  e_i2  l_i2  k_i2  reg2  trip_id
0     12    41     2    77    17    55    -2   123       16
1     38    57     3    94    45    73    -3   123       17
2     29    49     1   153    46    75    -1   123       18
3      1    23     3   126     9    40    -3   123       19
4      2    28     2    50    20    55    -2   123       20
5     39    67     2   152    59    96    -2   123       21
6      2    21     3   118    11    39    -3   123       22
7     45    55     1   154    54    73    -1   123       23
8      2    20     2    70    13    40    -2   123       24
9     15    24     1    49    27    45    -1   123       25
10     1    17     1    48    19    44    -1   123       26
11    13    30     2   162    27    53    -2   123       27
12     2    29     3   153    14    50    -3   123       28
13     6    32     2    49    15    50    -2   123       29
14     9    42     2    24    17    59    -2   123       30
[[0, 16, 46, 61], [0, 17, 47, 61], [0, 1

In [261]:
g_initial = initial(df)
print(g_initial)

[[0, 1, 31, 61], [0, 2, 32, 61], [0, 3, 33, 61], [0, 4, 34, 61], [0, 5, 35, 61], [0, 6, 36, 61], [0, 7, 37, 61], [0, 8, 38, 61], [0, 9, 39, 61], [0, 10, 40, 61], [0, 11, 41, 61], [0, 12, 42, 61], [0, 13, 43, 61], [0, 14, 44, 61], [0, 15, 45, 61], [0, 16, 46, 61], [0, 17, 47, 61], [0, 18, 48, 61], [0, 19, 49, 61], [0, 20, 50, 61], [0, 21, 51, 61], [0, 22, 52, 61], [0, 23, 53, 61], [0, 24, 54, 61], [0, 25, 55, 61], [0, 26, 56, 61], [0, 27, 57, 61], [0, 28, 58, 61], [0, 29, 59, 61], [0, 30, 60, 61]]


In [241]:
#preprocessing
def preprocessing(g):
    merge_candidate = []
    for i in range(0,len(g)):
        for j in range(0, len(g)):
            if g[i][1] != g[j][1]:
                if e_p[g[i][1]] + s_pd[g[i][1]]+tij[g[i][1],g[j][1]]<=l_p[g[j][1]]:
                    if q_p[g[i][1]] + q_p[g[j][1]] <= Qmax:
                        merge_candidate.append([g[i][1],g[j][1]])
                        
    return merge_candidate

In [242]:
def new_match_com(old_match, merge):
    new_match = []
    for j in range(len(old_match)):
        if merge[0] not in old_match[j] and merge[1] not in old_match[j]:
            new_match.append(old_match[j])
    
    new_tour = [0, merge[0], merge[1], merge[0]+30, 61]
    new_match.append([0, merge[0], merge[1], merge[0]+30, 61])
    
    return new_tour, new_match

In [243]:
def obtain_final_candidate(merge_candidate, g_initial):
    new_match_all = []
    new_tour_all = []
    merge_final = []
    
    for i in merge_candidate:
        new_tour_temp, new_match_temp = new_match_com(g_initial, i)
        if compute_time(new_tour_temp[0:-1]) != 'infeasible!':
            new_tour_all.append(new_tour_temp)
            new_match_all.append(new_match_temp)
            merge_final.append(i)
            
    return new_tour_all, new_match_all, merge_final

In [245]:
def compute_savings(new_match_all, old_match):
    savings = []
    for i in new_match_all:
        temp = obj_1(old_match)-obj_1(i)+obj_2(old_match)-obj_2(i)+obj_3(old_match)-obj_3(i)
        savings.append(temp)
    
    return savings
        
    

In [247]:
def del_merge_final(merge_final, merge):
    merge_candidate_new = []
    for i in merge_final:
        if (i[1] == merge[0] and i[0] != merge[1]) or (i[0] == merge[1] and i[1] != merge[0]):
            merge_candidate_new.append(i)
            
    return merge_candidate_new

In [249]:
def obtain_merge_tour(old_match,merge,new_tour_o):
    new_match = []
    for j in range(len(old_match)):
        if merge[0] not in old_match[j] and merge[1] not in old_match[j]:
            new_match.append(old_match[j])
    
    new_tour = []
    if merge[0] not in new_tour_o:
        new_tour.append(0)
        new_tour.append(merge[0])
        for m in range(len(new_tour_o)-1):
            new_tour.append(new_tour_o[m+1])
    if merge[1] not in new_tour_o:
        for m in range(len(new_tour_o)):
            new_tour.append(new_tour_o[m])
            if new_tour_o[m] == merge[0]:
                new_tour.append(merge[1])
            
    new_match.append(new_tour)
    
    return new_tour, new_match

In [250]:
def obtain_final_candidate_gen(merge_candidate, old_match,new_tour_o):
    new_match_all = []
    new_tour_all = []
    merge_final = []
    
    for i in merge_candidate:
        new_tour_temp, new_match_temp = obtain_merge_tour(old_match,i,new_tour_o)
#         print(new_tour_temp, new_match_temp)
        if compute_time(new_tour_temp[0:-1]) != 'infeasible!':
            new_tour_all.append(new_tour_temp)
            new_match_all.append(new_match_temp)
            merge_final.append(i)
            
    return new_tour_all, new_match_all, merge_final

In [291]:
def merge(g):
    g_initial = initial(g)
    merge_candidate_g = preprocessing(g_initial)
#     print(merge_candidate_g)
#     print(g_initial)
    new_tour_all, new_match_all, merge_final = obtain_final_candidate(merge_candidate_g, g_initial)
    print(merge_final)
#     print(new_match_all[9])
    
    merge_candidate_new = ["!"]
    savings_all = compute_savings(new_match_all,g_initial)
    index_candidate = sorted(range(len(savings_all)), key=lambda k: savings_all[k], reverse = True)
    print(savings_all)
    print(index_candidate)
    
    merge = merge_final[savings_all.index(max(savings_all))]
#     print(merge)
    old_match = new_match_all[savings_all.index(max(savings_all))]
#     print(old_match)
    new_tour_o = new_tour_all[savings_all.index(max(savings_all))]
#     print(old_match, new_tour_o)
    
    if len(merge_candidate_new):
        merge_candidate_new = del_merge_final(merge_final, merge)
        new_tour_all, new_match_all, merge_final = obtain_final_candidate_gen(merge_candidate_new, old_match,new_tour_o)
    
        savings_all = compute_savings(new_match_all,old_match)
        merge = merge_final[savings_all.index(max(savings_all))]
        old_match = new_match_all[savings_all.index(max(savings_all))]
        new_tour_o = new_tour_all[savings_all.index(max(savings_all))]
        
#         print(old_match, new_tour_o)
        
    return merge_final, old_match,new_tour_o
        

    

In [257]:
#g1 result
old_match_1, new_tour_o_1 = merge(g_1)
print(old_match_1)
print(new_tour_o_1)

[[2, 1], [2, 4], [3, 1], [3, 2], [3, 4], [3, 5], [4, 1], [5, 2], [5, 3], [5, 4]]
[64, 64, 64, 70, 64, 1, 66, 34, 10, 79]
[9, 3, 6, 0, 1, 2, 4, 7, 8, 5]
[[0, 2, 32, 61], [0, 3, 33, 61], [0, 5, 4, 1, 35, 61]]
[0, 5, 4, 1, 35, 61]


In [282]:
#g2 result
old_match_2, new_tour_o_2 = merge(g_2)
print(old_match_2)
print(new_tour_o_2)

[[7, 6], [7, 8], [7, 9], [7, 11], [7, 12], [9, 11], [10, 6], [10, 12], [11, 8], [11, 9], [11, 12], [12, 6], [12, 8], [12, 9], [12, 10], [12, 11], [12, 14], [14, 12], [15, 6], [15, 12]]
[16, 52, 24, 9, 29, 19, 42, 22, -8, 68, 33, 30, 24, 12, -2, 1, -41, 20, 35, 18]
[9, 1, 6, 18, 10, 11, 4, 2, 12, 7, 17, 5, 19, 0, 13, 3, 15, 14, 8, 16]
[[0, 6, 36, 61], [0, 7, 37, 61], [0, 8, 38, 61], [0, 10, 40, 61], [0, 13, 43, 61], [0, 14, 44, 61], [0, 15, 45, 61], [0, 12, 11, 9, 41, 61]]
[0, 12, 11, 9, 41, 61]


In [292]:
#g3 result
merge_final, old_match_3, new_tour_o_3 = merge(g_3)
print(merge_final)
print(old_match_3)
print(new_tour_o_3)

[[16, 18], [16, 20], [16, 23], [16, 29], [16, 30], [18, 21], [18, 23], [20, 16], [20, 18], [20, 21], [20, 23], [20, 24], [20, 25], [20, 26], [20, 29], [20, 30], [21, 18], [21, 23], [22, 18], [22, 23], [24, 18], [24, 21], [24, 23], [24, 25], [24, 26], [24, 29], [24, 30], [25, 18], [25, 20], [25, 23], [25, 29], [25, 30], [26, 18], [26, 20], [26, 21], [26, 23], [26, 24], [26, 25], [26, 28], [26, 29], [26, 30], [28, 18], [28, 23], [29, 18], [29, 20], [29, 21], [29, 23], [29, 24], [29, 25], [29, 26], [29, 30], [30, 18], [30, 21], [30, 23], [30, 24], [30, 29]]
[40, 35, 49, 23, 3, 72, 64, 13, 54, 47, 51, 1, 49, 44, 30, 15, 54, 51, 67, 68, 64, 77, 65, 24, 23, 5, 2, 53, 31, 50, 31, 13, 60, 17, 59, 57, 7, 36, 18, 17, 12, 73, 74, 58, 26, 55, 55, 9, 45, 40, 14, 70, 71, 67, 13, 3]
[21, 42, 41, 5, 52, 51, 19, 18, 53, 22, 6, 20, 32, 34, 43, 35, 45, 46, 8, 16, 27, 10, 17, 29, 2, 12, 9, 48, 13, 0, 49, 37, 1, 28, 30, 14, 44, 23, 3, 24, 38, 33, 39, 15, 50, 7, 31, 54, 40, 47, 36, 25, 4, 55, 26, 11]
[[21, 